In [13]:
import requests
import pandas as pd
import time

def fetch_courses(limit_per_page=100, max_pages=5):
    base_url = "https://api.coursera.org/api/courses.v1"
    
    fields = "name,description,slug,level,primaryLanguages,workload,domainTypes,certificates"
    
    all_courses = []
    start = 0
    page_count = 0
    
    print(f"Starting fetch process...")
    
    while page_count < max_pages:
        params = {
            "start": start,
            "limit": limit_per_page,
            "fields": fields
        }
        
        try:
            response = requests.get(base_url, params=params)
            
            if response.status_code == 200:
                data = response.json()
                elements = data.get('elements', [])
                
                if not elements:
                    print("No more data available.")
                    break
                
                filtered_count = 0
                for item in elements:
                    languages = item.get("primaryLanguages", [])
                    if 'en' not in languages:
                        continue

                    domains = item.get("domainTypes", [])
                    if domains:
                        category = domains[0].get("subdomainId") or domains[0].get("domainId") or "General"
                    else:
                        category = "General"

                    certs = item.get("certificates", [])
                    cert_str = ", ".join(certs) if certs else "Standard Course Certificate"

                    course_info = {
                        "id": item.get("id"),
                        "title": item.get("name"),
                        "description": item.get("description"),
                        "level": item.get("level", "Not Specified"),
                        "duration": item.get("workload", "Self-paced"),
                        "category": category,
                        "certificate_type": cert_str,
                        "url": f"https://www.coursera.org/learn/{item.get('slug')}"
                    }
                    all_courses.append(course_info)
                    filtered_count += 1
                
                print(f"Page {page_count + 1}: Kept {filtered_count} courses.")
                
                if 'paging' in data and 'next' in data['paging']:
                    start = int(data['paging']['next'])
                    page_count += 1
                else:
                    break
                
                time.sleep(1)
                
            else:
                print(f"Error: {response.status_code}")
                break
                
        except Exception as e:
            print(f"Exception: {e}")
            break

    return all_courses

def save_to_csv(courses, filename="coursera_dataset.csv"):
    if not courses:
        print("No data to save.")
        return

    df = pd.DataFrame(courses)
    df = df.dropna(subset=['description']) 
    
    if 'category' in df.columns:
        df['category'] = df['category'].str.replace('-', ' ').str.title()

    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Saved {len(df)} courses to {filename}")

if __name__ == "__main__":
    courses = fetch_courses(max_pages=5)
    save_to_csv(courses)

Starting fetch process...
Page 1: Kept 79 courses.
Page 2: Kept 76 courses.
Page 3: Kept 70 courses.
Page 4: Kept 78 courses.
Page 5: Kept 86 courses.
Saved 389 courses to coursera_dataset.csv


In [ ]:
import pandas as pd

# 1. อ่านไฟล์ CSV
df = pd.read_csv("coursera_dataset.csv")

pd.set_option('display.max_colwidth', 50)  # ให้โชว์แค่ 50 ตัวอักษรพอ เดี๋ยวล้นจอ
df

In [ ]:
pip install langchain-chroma langchain-huggingface chromadb

In [ ]:
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
import os

def build_database(csv_path="coursera_dataset.csv", db_path="./vector_store"):
    if not os.path.exists(csv_path):
        print(f"File not found: {csv_path}")
        return

    df = pd.read_csv(csv_path)
    
    # ใช้ Model ฟรีของ HuggingFace ทำงานบน CPU ได้ ไม่ต้องใช้ API Key
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    documents = []
    print(f"Processing {len(df)} courses...")

    for _, row in df.iterrows():
        # รวมข้อมูลที่จะให้ AI ใช้ค้นหาความหมาย
        content = f"""
        Title: {row['title']}
        Category: {row['category']}
        Level: {row['level']}
        Description: {row['description']}
        """
        
        # เก็บข้อมูลสำหรับนำไปแสดงผล (AI ไม่เอาไปคำนวณ แต่เก็บไว้ให้)
        metadata = {
            "id": str(row['id']),
            "title": row['title'],
            "url": row['url'],
            "duration": str(row['duration']),
            "certificate": str(row['certificate_type']),
            "level": str(row['level'])
        }

        doc = Document(page_content=content.strip(), metadata=metadata)
        documents.append(doc)

    print("Creating Vector Database...")
    
    # สร้าง DB และบันทึกลงโฟลเดอร์ (Persist)
    Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        persist_directory=db_path
    )
    
    print(f"Database saved to {db_path}")

def test_search(query, db_path="./vector_store"):
    print(f"\nTesting Search: '{query}'")
    
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = Chroma(persist_directory=db_path, embedding_function=embedding_model)
    
    # ค้นหา 3 อันดับแรกที่ใกล้เคียงที่สุด
    results = db.similarity_search(query, k=3)
    
    for i, doc in enumerate(results):
        print(f"{i+1}. {doc.metadata['title']} ({doc.metadata['level']})")
        print(f"   Link: {doc.metadata['url']}")
        print("-" * 20)

if __name__ == "__main__":
    # 1. สร้าง DB
    build_database()
    
    # 2. ลองค้นหาดู
    test_search("I want to learn about Python for Data Science")

c:\Users\snails\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\snails\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\snails\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to

Processing 389 courses...
Creating Vector Database...
Database saved to ./vector_store

Testing Search: 'I want to learn about Python for Data Science'
1. Python Programming Fundamentals (BEGINNER)
   Link: https://www.coursera.org/learn/microsoft-python-programming-fundamentals
--------------------
2. Intermediate Python – Libraries, Tools & Practical Projects (INTERMEDIATE)
   Link: https://www.coursera.org/learn/packt-intermediate-python-libraries-tools-and-practical-projects-0d9as
--------------------
3. Data Visualization and Modeling in Python (INTERMEDIATE)
   Link: https://www.coursera.org/learn/python-data-modeling
--------------------


In [4]:
test_search("i want to learn TailwindCSS")


Testing Search: 'i want to learn TailwindCSS'
1. Tailwind CSS Practice Project: Build a Product Card (INTERMEDIATE)
   Link: https://www.coursera.org/learn/build-a-product-card-with-tailwind-css
--------------------
2. Create a Dark Moody Atmospheric 2D Game with Unity and C# (BEGINNER)
   Link: https://www.coursera.org/learn/packt-create-a-dark-moody-atmospheric-2d-game-with-unity-and-c-2qqm9
--------------------
3. Unix System Overview and Command (INTERMEDIATE)
   Link: https://www.coursera.org/learn/unix-system-overview-and-command
--------------------


In [11]:
import os

try:
    current_path = os.path.dirname(os.path.abspath(__file__))
except NameError:
    current_path = os.getcwd() # กันเหนียวเผื่อรันใน Notebook

project_root = current_path
    
    # 2. วนลูปเดินถอยหลังขึ้นไปเรื่อยๆ จนกว่าจะเจอโฟลเดอร์ชื่อ 'data'
while True:
    possible_data_path = os.path.join(project_root, 'data')
    
    if os.path.exists(possible_data_path):
        # เจอแล้ว! หยุดตรงนี้แหละคือ Root
        print(f"✅ Found project root at: {project_root}")
        break
    
    # ถ้าไม่เจอ ให้ถอยหลังขึ้นไปอีก 1 ชั้น
    parent_dir = os.path.dirname(project_root)
    
    if parent_dir == project_root:
        # ถ้าถอยจนสุดทาง (C:\) แล้วยังไม่เจอ แสดงว่าวางไฟล์ผิดที่
        print("❌ Error: Could not find 'data' folder.")
        print("Please check if 'data' folder exists inside your project.")
        
    project_root = parent_dir

    # 3. กำหนด path ที่ถูกต้องจาก Root ที่หาเจอ
csv_path = os.path.join(project_root, 'data', 'coursera_dataset.csv')
db_path = os.path.join(project_root, 'vector_store')

✅ Found project root at: c:\SUPERPROJECT


In [ ]:
import requests
import pandas as pd
import time
import re
import random # เพิ่ม random มาช่วยสุ่มเวลา

def remove_html_tags(text):
    if not text:
        return ""
    clean = re.compile('<.*?>')
    return re.sub(clean, ' ', text).strip()

def fetch_skilllane_data(limit=100):
    all_courses = []
    
    # User-Agent สำคัญมาก ต้องใส่ให้เหมือน Browser จริง
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Referer": "https://www.skilllane.com/" # ใส่ Referer หลอกว่ามาจากหน้าเว็บเขา
    }

    print(f"🚀 Fetching SkillLane Single Courses (Limit: {limit})...")
    
    offset = 0
    batch_size = 15 # API ตัวนี้บังคับทีละ 15
    
    while len(all_courses) < limit:
        url = "https://b2c-api.skilllane.com/v1/courses"
        params = {
            "limit": batch_size, 
            "offset": offset
        }
        
        try:
            resp = requests.get(url, params=params, headers=headers)
            
            # --- Anti-Ban Logic ---
            if resp.status_code == 429:
                print("⚠️ โดน Rate Limit (ยิงเร็วไป)! หยุดพัก 60 วินาที...")
                time.sleep(60)
                continue # ลองใหม่
                
            if resp.status_code == 403:
                print("❌ โดนบล็อก IP (Forbidden)! จบการทำงานทันที")
                break
                
            if resp.status_code != 200: 
                print(f"❌ Error: {resp.status_code}")
                break
            # ----------------------
            
            items = resp.json().get('items', [])
            if not items: 
                print("✅ หมดข้อมูลแล้ว")
                break
            
            for item in items:
                # Logic เดิม
                instructors = item.get('instructors_info', [])
                inst_name = "SkillLane Instructor"
                inst_title = ""
                if instructors:
                    fname = instructors[0].get('firstname', '').strip()
                    lname = instructors[0].get('lastname', '').strip()
                    inst_name = f"{fname} {lname}"
                    inst_title = instructors[0].get('highlight_desc', '')

                desc = remove_html_tags(item.get('description', ''))
                
                course_info = {
                    "id": f"skilllane_{item.get('id')}",
                    "title": item.get('name'),
                    "description": desc,
                    "instructor": f"{inst_name} ({inst_title})".strip(),
                    "price": f"{item.get('price', 0)} THB",
                    "duration": f"{item.get('duration', 0) // 60} mins",
                    "category": "General", # SkillLane ไม่ส่ง category มาในหน้านี้ ต้องทำใจ
                    "url": f"https://www.skilllane.com/courses/{item.get('id_name')}",
                    "source": "SkillLane"
                }
                all_courses.append(course_info)
            
            print(f"   Fetched {len(items)} courses... (Total: {len(all_courses)})")
            
            offset += batch_size
            
            # [Safe Mode] สุ่มเวลาพัก เพื่อให้ดูเหมือนคนกดเปลี่ยนหน้า
            sleep_time = random.uniform(1.5, 3.5) 
            time.sleep(sleep_time)
            
        except Exception as e:
            print(f"❌ Exception: {e}")
            break

    return all_courses[:limit] # ตัดให้พอดีจำนวนที่ขอ

def save_to_csv(courses, filename="skilllane_dataset.csv"):
    if not courses: return
    df = pd.DataFrame(courses)
    df = df.dropna(subset=['description'])
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ Saved {len(df)} items to {filename}")

if __name__ == "__main__":
    data = fetch_skilllane_data(limit=100)
    save_to_csv(data)

🚀 Fetching SkillLane Single Courses (Limit: 100)...
   Fetched 15 courses... (Total: 15)
   Fetched 15 courses... (Total: 30)
   Fetched 15 courses... (Total: 45)
   Fetched 15 courses... (Total: 60)
   Fetched 15 courses... (Total: 75)
   Fetched 15 courses... (Total: 90)
   Fetched 15 courses... (Total: 105)
✅ Saved 100 items to skilllane_dataset.csv


In [7]:
pd.read_csv("skilllane_dataset.csv")

,id,title,description,instructor,price,duration,category,url,source
0,skilllane_7743,HPT : High-Performing Team สร้างทีมสำเร็จสูง,คำอธิบายคอร์สออนไลน์ ในโลกการทำงานทุกวันนี้ ห...,อิศรา สมิตะพินทุ (อ.เอ้) (ผู้ก่อตั้งบริษัท อิน...,4990 THB,84 mins,General,https://www.skilllane.com/courses/high-perform...,SkillLane
1,skilllane_7860,n8n AI Agent & OCR Automation สำหรับฝ่ายบัญชี ...,คอร์สนี้ออกแบบมาเพื่อให้คุณสร้างระบบอัตโนมัติท...,ดร.อมรเทพ ทองชิว (นักวิจัยข้อมูล วิทยากรด้านกา...,0 THB,243 mins,General,https://www.skilllane.com/courses/n8n-OCR-Auto...,SkillLane
2,skilllane_3981,AutoCAD เขียนแบบเครื่องกลเบื้องต้น,คำอธิบายคอร์สออนไลน์ AutoCAD เป็นโปรแกรมที่สา...,รัฐวิสุทธิ์ กล่อมจิตต์ (วิทยากรและที่ปรึกษาทาง...,2990 THB,842 mins,General,https://www.skilllane.com/courses/AutoCAD-for-...,SkillLane
3,skilllane_7726,สร้างโมเดล 3D อย่างมืออาชีพ ด้วย AutoCAD ฉบับเ...,คำอธิบายคอร์สออนไลน์ คอร์ส สร้างโมเดล 3 มิติ ...,รัฐวิสุทธิ์ กล่อมจิตต์ (วิทยากรและที่ปรึกษาทาง...,1490 THB,470 mins,General,https://www.skilllane.com/courses/AutoCAD-3D-b...,SkillLane
4,skilllane_7221,เคล็ดลับออมเงิน ฉบับคนเริ่มทำงาน,สร้างความรู้ความเข้าใจเกี่ยวกับแนวทางการวางแผน...,SET Instructor (None),0 THB,66 mins,General,https://www.skilllane.com/courses/SET-WMD1012,SkillLane
...,...,...,...,...,...,...,...,...,...
95,skilllane_7674,"สอนทำ ""ก๋วยเตี๋ยวราดหน้าหมูหมัก"" สูตรมาตรฐาน-ส...",คำอธิบายคอร์สออนไลน์ ห้ามพลาด!! คอร์ส สอนทำ ...,ธนะศักดิ์ พาหุจินดา (เชฟแดน) (Food Consultant ...,999 THB,29 mins,General,https://www.skilllane.com/courses/stir-fried-n...,SkillLane
96,skilllane_7669,สอนสร้าง Template ระบบสต๊อกสินค้าใน Google Sheets,คำอธิบายคอร์ส คอร์สนี้สอนการสร้าง “ระบบสต๊อกส...,"วงศกร นิมมิต (เจ้าของเพจ ""บริหารงานแบบคนไอที"" ...",0 THB,166 mins,General,https://www.skilllane.com/courses/GoogleSheets...,SkillLane
97,skilllane_3879,คอร์สเรียนไวยากรณ์เกาหลี ระดับ 4 ม.โซล (เล่มSn...,คำอธิบายคอร์สออนไลน์ คอร์สเรียนไวยากรณ์เกาหลี...,ธิดา พัชรพรพงศ์ (ครูสอนภาษาเกาหลี และนักศึกษาท...,6500 THB,239 mins,General,https://www.skilllane.com/courses/Korean-Gramm...,SkillLane
98,skilllane_7622,Notebook LM 101: เคล็ดลับใช้ AI เพิ่ม Producti...,คำอธิบายคอร์สออนไลน์ Notebook LM 101: เคล็ดลับ...,ภควดี สมพงษ์ (เจ้าของเพจ SPEAK SPARK - present...,2500 THB,175 mins,General,https://www.skilllane.com/courses/NotebookLM-101,SkillLane
